# Intent Classification with ThirdAI's Universal Deep Transformer
This notebook shows how to build an intent classification model with ThirdAI's Universal Deep Transformer (UDT) model, our all-purpose classifier for tabular datasets. In this demo, we will train and evaluate the model on the CLINC 150 dataset, but you can easily replace this with your own dataset.

To run this notebook, you will need to obtain a ThirdAI license at the following link if you have not already: https://www.thirdai.com/try-bolt/

In [ ]:
!pip3 install -r requirements.txt
!pip3 install thirdai --upgrade

# Dataset Download
We will use the utils module in this repo to download dataset_name (if you have just copied this notebook and not cloned the entire repo, you will need to copy the utils.py file as well). You can replace this step and the next step with a download method and a UDT initialization step that is specific to your dataset.

In [ ]:
import utils

train_filename, test_filename, inference_batch = utils.download_clinc()

# UDT Initialization
We can now create a UDT model by passing in the types of each column in the dataset and the target column we want to be able to predict.

In [ ]:
from thirdai import bolt

model = bolt.UniversalDeepTransformer(
    data_types={
        "text": bolt.types.text(),
        "category": bolt.types.categorical(),
    },
    target="category",
    n_target_classes=151,
)

# Training
We can now train our UDT model with just two lines! Feel free to customize the number of epochs and the learning rate; we have chosen values that give good convergence.

In [ ]:
model.train(train_filename, epochs=5, learning_rate=0.01, metrics=["categorical_accuracy"])

# Evaluation
Evaluating the performance of the UDT model is also just two lines!

In [ ]:
model.evaluate(test_filename, metrics=["categorical_accuracy"]);

# Saving and Loading
Saving and loading a trained UDT model to disk is also extremely straight forward.

In [ ]:
save_location = "intent_classification.model"

# Saving
model.save(save_location)

# Loading
model = bolt.UniversalDeepTransformer.load(save_location)

# Testing Predictions
The evaluation method is great for testing, but it requires labels, which don't exist in a production setting. We also have a predict method that can take in an in-memory batch of rows or a single row (without the target column), allowing easy integration into production pipelines.

In [ ]:
import numpy as np

inference_batch = inference_batch[:5]
inference_batch_x = [element[0] for element in inference_batch]
print("Inference batch:", inference_batch_x, "\n")

prediction = model.predict(inference_batch_x[0])
class_name = model.class_name(np.argmax(prediction))
print("Input:", inference_batch_x[0], "Prediction:", class_name, "\n")

prediction_batch = model.predict_batch(inference_batch_x)
class_names = [
    model.class_name(class_id) for class_id in np.argmax(prediction_batch, axis=1)
]
print("Batch Prediction Results")
for input_sample, class_name in zip(inference_batch, class_names):
    print("Input:", input_sample, "Prediction:", class_name)